<a href="https://colab.research.google.com/github/Sanjaygandhari/Autism_DL/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import zipfile
import pandas as pd

# Step 1: Define the path of the uploaded zip file and extraction path
zip_path = '/content/consolidated.zip'  # Adjust path if needed
extract_path = '/content'  # Path where the zip will be extracted

# Step 2: Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)  # Extracts to /content/consolidated

# Step 3: Initialize lists to store file paths and labels
file_paths = []
labels = []

# Step 4: Loop through each subfolder (category) within the consolidated folder
consolidated_path = os.path.join(extract_path, 'consolidated')
for category in ['autistic', 'non_autistic']:
    category_path = os.path.join(consolidated_path, category)  # Full path to the subfolder

    # Ensure the category path exists to avoid errors
    if os.path.isdir(category_path):
        # Loop through each file in the category subfolder
        for filename in os.listdir(category_path):
            # Only add valid image files (e.g., .jpg, .png, .jpeg)
            if filename.endswith(('.jpg', '.png', '.jpeg')):
                file_paths.append(os.path.join(category_path, filename))  # Full file path
                labels.append(category)  # Use the folder name as the label

# Step 5: Create a DataFrame with the collected file paths and labels
train_df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

# Step 6: Display the DataFrame with only the file paths and labels
print(train_df[['file_path', 'label']])


                                        file_path         label
0          /content/consolidated/autistic/465.jpg      autistic
1           /content/consolidated/autistic/49.jpg      autistic
2          /content/consolidated/autistic/953.jpg      autistic
3          /content/consolidated/autistic/994.jpg      autistic
4          /content/consolidated/autistic/755.jpg      autistic
...                                           ...           ...
2921  /content/consolidated/non_autistic/1092.jpg  non_autistic
2922   /content/consolidated/non_autistic/803.jpg  non_autistic
2923  /content/consolidated/non_autistic/1107.jpg  non_autistic
2924  /content/consolidated/non_autistic/1314.jpg  non_autistic
2925  /content/consolidated/non_autistic/1028.jpg  non_autistic

[2926 rows x 2 columns]


In [6]:
import tensorflow as tf  # Import TensorFlow

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator( # Use 'tf' instead of 'tensorflow'
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,  # Use 'tf'
    validation_split=0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator( # Use 'tf' instead of 'tensorflow'
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,  # Use 'tf'
    rescale=1./255,
)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming 'train_df' was created or loaded earlier
# Split into train, validation, and test sets
train, temp = train_test_split(train_df, test_size=0.3, random_state=42)  # Split into train and temporary (test + validation)
valid, test = train_test_split(temp, test_size=0.5, random_state=42)      # Split temporary into validation and test

# Rename the 'label' column to 'label_name' for consistency
train = train.rename(columns={'label': 'label_name'})
valid = valid.rename(columns={'label': 'label_name'})
test = test.rename(columns={'label': 'label_name'})

# ImageDataGenerator for augmentation (customize as needed)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,       # Rescale pixel values
    rotation_range=20,     # Randomly rotate images
    width_shift_range=0.2, # Randomly shift images horizontally
    height_shift_range=0.2 # Randomly shift images vertically
)

test_datagen = ImageDataGenerator(rescale=1.0/255)  # Only rescaling for validation and test sets

# Prepare generators
train_imgs = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col='file_path',
    y_col='label_name',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
)

valid_imgs = test_datagen.flow_from_dataframe(  # Use test_datagen for validation
    dataframe=valid,
    x_col='file_path',
    y_col='label_name',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
)

test_imgs = test_datagen.flow_from_dataframe(  # Use test_datagen for the test set
    dataframe=test,
    x_col='file_path',
    y_col='label_name',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=20,
    shuffle=False  # No shuffling for test set to maintain order
)


Found 2048 validated image filenames belonging to 2 classes.
Found 439 validated image filenames belonging to 2 classes.
Found 439 validated image filenames belonging to 2 classes.


In [8]:
from sklearn.model_selection import train_test_split

# Assuming train_df was created in the previous step
train, test = train_test_split(train_df, test_size=0.2, random_state=42)

In [9]:
# Get class names from the 'train_imgs' generator
class_names = list(train_imgs.class_indices.keys())

# Display the class names
print(class_names)


['autistic', 'non_autistic']


In [10]:
#Part 2 Building the CNN
#Initialising the CNN
cnn = tf.keras.models.Sequential()

#Step 1 Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))

#Step 2 Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Step 3 Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Step 5 Output Layer
#cnn.add(tf.keras.layers. Dense(units-1, activation 'sigmoid'))
##For Binary Classification
# Use tf.keras.layers.Dense to define the Dense layer
cnn.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='linear'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

# Initialize a Sequential model
model = Sequential()

# Add an input layer with the specified shape (adjust as per your images)
model.add(Input(shape=(224, 224, 3)))  # 224x224 with 3 channels (RGB)

# Add convolutional and pooling layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

# Add a fully connected layer and output layer
# Changed to 2 units and softmax activation for binary classification
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Modified output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks1= [EarlyStopping(monitor='val_accuracy',
min_delta=0,
patience=2,
mode='auto')]

# Fit the model
# Change 'valid_gen' to 'valid_imgs'
history = model.fit(train_imgs, validation_data=valid_imgs, epochs=40, callbacks=callbacks1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
103/103 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.5434 - loss: 0.9464 - val_accuracy: 0.6241 - val_loss: 0.6686
Epoch 2/40
103/103 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.6194 - loss: 0.6756 - val_accuracy: 0.6720 - val_loss: 0.6511
Epoch 3/40
103/103 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.6378 - loss: 0.6600 - val_accuracy: 0.6538 - val_loss: 0.6397
Epoch 4/40
103/103 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.6556 - loss: 0.6440 - val_accuracy: 0.6606 - val_loss: 0.6281


In [12]:
model.save('/content/my_model.h5')

In [13]:
# Testing the model
results = model.evaluate(test_imgs, verbose=0)
print("Test Loss : {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Loss : 0.62325
Test Accuracy: 66.29%


In [17]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [21]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
# Load the trained model
@st.cache(allow_output_mutation=True)  # Cache the model to avoid reloading on every run
def load_model():
    model = tf.keras.models.load_model("/content/my_model.h5")  # Provide the path to your saved model
    return model

model = load_model()
# Function to preprocess and make predictions
def preprocess_and_predict(image_data, model):
    # Convert image to RGB (if needed)
    image = Image.open(image_data).convert("RGB")
    # Resize the image to (224, 224) as expected by the model
    image = image.resize((224, 224))
    # Convert the image to a numpy array
    image_array = np.asarray(image) / 255.0  # Normalize pixel values to [0, 1]
    # Add batch dimension
    image_array = np.expand_dims(image_array, axis=0)
    # Make prediction
    prediction = model.predict(image_array)
    return prediction
# Streamlit UI
st.title("Prediction of Autism Spectrum Condition")
st.text("Please upload an image file:")
# File uploader
uploaded_file = st.file_uploader("Choose an image", type=["jpg", "png", "jpeg"])
if uploaded_file is not None:
    # Display the uploaded image
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    st.write("Classifying...")

    # Preprocess and predict
    predictions = preprocess_and_predict(uploaded_file, model)
    score = tf.nn.softmax(predictions[0])  # Apply softmax to interpret as probabilities

    # Define class names
    class_names = ['autistic', 'non_autistic']  # Replace with actual class names
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    # Display prediction
    st.success(f"This image is most likely: **{predicted_class}** with a confidence of {confidence:.2f}%")


Writing app.py


In [22]:
!pip install pyngrok
from pyngrok import ngrok

In [27]:
!ngrok config add-authtoken 2p273CQLUHmKf5jEkn8cfT4YkAa_7tySqXBr6GcxqoehNs4na

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [24]:
!pip install scikit-learn


In [28]:
from pyngrok import ngrok
import os

ngrok.set_auth_token("2p273CQLUHmKf5jEkn8cfT4YkAa_7tySqXBr6GcxqoehNs4na")
# Run Streamlit app in the background
os.system("streamlit run app.py &")

# Open a tunnel on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Streamlit app is running at: NgrokTunnel: "https://4bfb-34-81-208-107.ngrok-free.app" -> "http://localhost:8501"
